In [27]:
import glob
import hashlib
import os
import re
import subprocess
from concurrent.futures import ThreadPoolExecutor

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from moviepy.editor import VideoFileClip
from sklearn.metrics import f1_score
from tqdm import tqdm

In [5]:
# Загрузка данных
data = pd.read_csv("train.csv")

In [ ]:
# Первые 5 строк
print(data.head())

# Информация о датасете
print(data.info())

In [ ]:
# Проверка на полные дубликаты
duplicates = data.duplicated().sum()
print(f"Количество полных дубликатов: {duplicates}")

In [ ]:
# Проверка на пропущенные значения
missing_values = data.isna().sum()
print(missing_values[missing_values > 0])

In [ ]:
# Проверка типов данных
print(data.dtypes)

# Приведение 'created' к типу datetime
data["created"] = pd.to_datetime(data["created"])

In [ ]:
# Распределение целевой переменной


sns.countplot(x="is_duplicate", data=data)
plt.title("Распределение дубликатов")
plt.xlabel("Является дубликатом")
plt.ylabel("Количество")
plt.show()

In [ ]:
# Распределение сложных кейсов
sns.countplot(x="is_hard", data=data)
plt.title("Распределение сложных кейсов")
plt.xlabel("Сложный кейс")
plt.ylabel("Количество")
plt.show()

In [ ]:
# Распределение по времени загрузки
data["created"].dt.date.value_counts().sort_index().plot(kind="bar", figsize=(12, 6))
plt.title("Количество загрузок видео по датам")
plt.xlabel("Дата")
plt.ylabel("Количество загрузок")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Количество дублей для каждого оригинала
duplicate_counts = data[data["is_duplicate"]]["duplicate_for"].value_counts()
plt.figure(figsize=(12, 6))
duplicate_counts.plot(kind="hist", bins=30)
plt.title("Количество дублей на оригинал")
plt.xlabel("Количество дублей")
plt.ylabel("Количество оригиналов")
plt.show()

### md5

In [ ]:
# Загрузка данных
data = pd.read_csv("updated_train_with_embeddings.csv")

# Путь к папке с видеофайлами
video_folder = "train_dataset"


# Функция для получения информации о видео
def get_video_info(index, video_uuid):
    video_path = os.path.join(video_folder, f"{video_uuid}.mp4")  # Предполагаем, что файлы названы по UUID с расширением .mp4
    try:
        # Проверяем, существует ли файл
        if not os.path.exists(video_path):
            print(f"Видео не найдено: {video_path}")
            return index, None, None, None

        # Получение длины видео и размера файла
        clip = VideoFileClip(video_path)
        duration = clip.duration  # Длительность в секундах
        size = os.path.getsize(video_path)  # Размер в байтах

        # Вычисление MD5
        md5_hash = hashlib.md5()
        # print(f"Чтение видео для MD5: {video_path}")
        with open(video_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                md5_hash.update(chunk)
        md5_value = md5_hash.hexdigest()

        return index, duration, size, md5_value
    except Exception as e:
        print(f"Ошибка при обработке {video_path}: {e}")
        return index, None, None, None


# Функция для обработки видео с параллелизацией
def process_videos_in_parallel(data, max_workers=32):
    # Список для хранения результатов
    results = []

    # Функция для передачи данных в потоки
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Используем tqdm для отображения прогресса
        futures = [executor.submit(get_video_info, index, row["uuid"]) for index, row in data.iterrows()]

        # tqdm отображает прогресс выполнения
        for future in tqdm(futures, desc="Processing videos"):
            result = future.result()
            results.append(result)

    # Обновление DataFrame с результатами
    for result in results:
        index, duration, size, md5_value = result
        data.at[index, "duration"] = duration
        data.at[index, "size"] = size
        data.at[index, "md5"] = md5_value


# Инициализация новых колонок
data["duration"] = None
data["size"] = None
data["md5"] = None

# Обработка видеофайлов параллельно с отображением прогресса
process_videos_in_parallel(data, max_workers=32)

# Сохранение обновленного DataFrame
data.to_csv("updated_train_emb_md5.csv", index=False)

In [16]:
data = pd.read_csv("updated_train_emb_md5.csv")

In [ ]:
# Анализ полученных данных

# Статистика по длительности видео
print("Статистика по длительности видео:")
print(data["duration"].describe())

# Статистика по размеру файлов
print("\nСтатистика по размеру файлов (в байтах):")
print(data["size"].describe())

# Анализ на наличие одинаковых MD5
duplicate_md5 = data[data.duplicated(subset="md5", keep=False)]
print(f"\nКоличество файлов с одинаковыми MD5: {len(duplicate_md5)}")

# Вывод всех файлов с одинаковыми MD5
if len(duplicate_md5) > 0:
    print("\nФайлы с одинаковыми MD5:")
    print(duplicate_md5[["uuid", "md5"]])

# Визуализация распределения длительности видео
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(data["duration"].dropna(), bins=30, color="blue", alpha=0.7)
plt.title("Распределение длительности видео")
plt.xlabel("Длительность (секунды)")
plt.ylabel("Количество видео")
plt.grid(True)
plt.show()

# Визуализация распределения размера файлов
plt.figure(figsize=(10, 6))
plt.hist(data["size"].dropna(), bins=30, color="green", alpha=0.7)
plt.title("Распределение размера файлов")
plt.xlabel("Размер файла (байты)")
plt.ylabel("Количество видео")
plt.grid(True)
plt.show()

### TMK emb

In [ ]:
# Загрузка данных
data = pd.read_csv("train.csv")

# Путь к папке с видеофайлами
video_folder = "train_dataset"

# Путь к исполняемым файлам TMK и FFmpeg
tmk_executable_path = "/jupyter/ThreatExchange/tmk/cpp"
ffmpeg_path = "/usr/bin/ffmpeg"  # Убедитесь, что путь корректен.

# Директория для сохранения эмбеддингов
embedding_directory = "/jupyter/emb"

# URL-шаблон для скачивания видео при отсутствии ссылки
video_url_template = "https://s3.ritm.media/yappy-db-duplicates/{uuid}.mp4"


# Функция для загрузки видеофайла по ссылке
def download_video(video_url, output_path):
    try:
        print(f"Загрузка видео с {video_url} в {output_path}")

        # Отправляем запрос на получение файла
        response = requests.get(video_url, stream=True, timeout=60)

        # Проверяем статус ответа
        if response.status_code == 200:
            # Открываем файл для записи
            with open(output_path, "wb") as f:
                for chunk in response.iter_content(1024 * 1024):  # Загружаем по 1 МБ
                    if chunk:
                        f.write(chunk)

            # Проверяем, действительно ли файл был успешно записан
            if os.path.exists(output_path):
                print(f"Видео успешно загружено: {output_path}")
                return output_path
            else:
                print(f"Ошибка при сохранении файла: {output_path}")
                return None
        else:
            print(f"Ошибка загрузки видео, статус код: {response.status_code} для {video_url}")
            return None

    except Exception as e:
        print(f"Ошибка при загрузке видео {video_url}: {e}")
        return None


# Функция для генерации эмбеддингов с использованием TMK
def generate_tmk_embedding(video_path):
    try:
        # Убедитесь, что директория для эмбеддингов существует
        if not os.path.exists(embedding_directory):
            os.makedirs(embedding_directory)

        # Извлекаем имя файла без расширения
        video_filename = os.path.basename(video_path)
        video_base_name = os.path.splitext(video_filename)[0]  # Получаем имя без .mp4

        # Формирование пути к сгенерированному файлу эмбеддинга
        tmk_output = os.path.join(embedding_directory, f"{video_base_name}.tmk")

        # Проверка, существует ли файл эмбеддинга
        if os.path.exists(tmk_output):
            return tmk_output  # Эмбеддинг уже существует, возвращаем его путь

        # Запуск TMK для создания эмбеддинга
        subprocess.run(
            [os.path.join(tmk_executable_path, "tmk-hash-video"), "-f", ffmpeg_path, "-i", video_path, "-o", tmk_output],
            check=True,
        )

        return tmk_output
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при генерации эмбеддинга для {video_path}: {e}")
        return None


# Функция для обработки одного видео (загрузка и создание эмбеддинга)
def process_video(uuid, video_link, retries=2):
    video_path = os.path.join(video_folder, f"{uuid}.mp4")

    for attempt in range(retries + 1):
        # Проверка, существует ли видео
        if not os.path.exists(video_path) or attempt > 0:
            if attempt > 0:
                print(f"Попытка #{attempt} загрузки {uuid}")
            # Если ссылка отсутствует, используем шаблон для загрузки видео
            if not video_link or pd.isna(video_link):
                video_link = video_url_template.format(uuid=uuid)
            # Если видео не найдено или произошла ошибка, загружаем его по ссылке
            video_path = download_video(video_link, video_path)
            if video_path is None:
                return None  # Пропускаем этот файл, если его не удалось загрузить

        # Генерация эмбеддинга для видео
        tmk_embedding_path = generate_tmk_embedding(video_path)

        if tmk_embedding_path:
            return tmk_embedding_path  # Успешно обработали

        # Если ошибка, удаляем поврежденное видео и пытаемся снова
        if os.path.exists(video_path):
            os.remove(video_path)
            print(f"Удалено поврежденное видео: {video_path}")

    return None  # Если все попытки не удались


# Функция для обработки основного видео и его дубликата
def process_both_videos(index, uuid, video_link, duplicate_uuid, data):
    # Обработка основного видео
    tmk_embedding_path = process_video(uuid, video_link)
    if tmk_embedding_path:
        data.at[index, "tmk_embedding"] = tmk_embedding_path

    # Обработка видео из duplicate_for
    if pd.notna(duplicate_uuid) and pd.isna(data.at[index, "duplicate_tmk_embedding"]):
        # Попробуем получить duplicate_video_link из data
        duplicate_row = data[data["uuid"] == duplicate_uuid]
        duplicate_video_link = duplicate_row.iloc[0]["link"] if not duplicate_row.empty else None

        # Обработка duplicate_video_link
        duplicate_tmk_embedding_path = process_video(duplicate_uuid, duplicate_video_link)
        if duplicate_tmk_embedding_path:
            data.at[index, "duplicate_tmk_embedding"] = duplicate_tmk_embedding_path
        else:
            print(f"Не удалось обработать duplicate_uuid {duplicate_uuid}")


# Основная функция для распараллеливания загрузки и обработки видео
def process_videos_in_parallel(data, max_workers=32):
    # Инициализация новых столбцов для эмбеддингов, если их еще нет
    if "tmk_embedding" not in data.columns:
        data["tmk_embedding"] = None
    if "duplicate_tmk_embedding" not in data.columns:
        data["duplicate_tmk_embedding"] = None

    # Создаем список задач для обработки
    tasks = []

    # Собираем данные для обработки
    for index, row in data.iterrows():
        uuid = row["uuid"]
        video_link = row.get("link", None)
        duplicate_uuid = row.get("duplicate_for", None)

        tasks.append((index, uuid, video_link, duplicate_uuid))

    # Прогресс бар
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for task in tasks:
            index, uuid, video_link, duplicate_uuid = task
            futures.append(executor.submit(process_both_videos, index, uuid, video_link, duplicate_uuid, data))

        # Обновление прогресс-бара
        for future in tqdm(futures, total=len(futures), desc="Processing videos"):
            future.result()  # Ожидаем завершения задачи


# Запуск параллельной обработки
process_videos_in_parallel(data, max_workers=32)

# Сохранение обновленного DataFrame с путями к эмбеддингам
data.to_csv("updated_train_with_embeddings.csv", index=False)

print("Генерация TMK-эмбеддингов завершена.")

### получаем кластера, ходим кластера 2 и больше, если в кластере больше 2 видео значит они сложные и ставим метку is_hard

In [25]:
# Шаг 1: Запуск команды tmk-clusterize-parallel и парсинг ее вывода
def run_tmk_clusterize():
    command = "find /jupyter/dup/emb -name '*.tmk' | " "/jupyter/ThreatExchange/tmk/cpp/tmk-clusterize-parallel " "-s --min 2 -i"

    # Запуск команды
    result = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

    # Получение вывода команды
    cluster_output = result.stdout
    return cluster_output


# Функция для парсинга данных из tmk-clusterize-parallel
def parse_tmk_cluster_output(cluster_output):
    clusters = {}
    for line in cluster_output.strip().splitlines():
        match = re.match(r"clidx=(\d+),clusz=(\d+),filename=\/[a-zA-Z0-9\/\.\-_]+\/([a-f0-9\-]+)\.tmk", line)
        if match:
            clidx = int(match.group(1))
            uuid = match.group(3)
            if clidx not in clusters:
                clusters[clidx] = []
            clusters[clidx].append(uuid)
    # print("Распарсенные кластеры:")
    # print(clusters)
    return clusters


# Шаг 2: Запуск tmk-clusterize-parallel и получение кластеров
cluster_output = run_tmk_clusterize()
clusters = parse_tmk_cluster_output(cluster_output)

In [ ]:
# Шаг 3: Чтение DataFrame с данными
df = pd.read_csv("updated_train_emb_md5.csv")

# Преобразуем колонку 'created' в datetime формат для корректного сравнения
df["created"] = pd.to_datetime(df["created"])

# Добавляем новые колонки
df["is_duplicate_tmk"] = False
df["duplicate_for_tmk"] = None
df["is_hard_tmk"] = False  # Добавляем колонку для сложных кластеров

# Обработка случаев, когда uuid есть в duplicate_for, но отсутствует в uuid
# Находим такие uuid и добавляем их в DataFrame
duplicate_for_uuids = df["duplicate_for"].dropna().unique()
existing_uuids = df["uuid"].unique()
missing_uuids = set(duplicate_for_uuids) - set(existing_uuids)

# Создаем DataFrame с отсутствующими оригиналами
if missing_uuids:
    missing_rows = pd.DataFrame({"uuid": list(missing_uuids)})
    # Заполняем недостающие колонки
    missing_rows["created"] = pd.NaT  # Нет данных о дате создания
    missing_rows["link"] = None
    missing_rows["is_duplicate"] = False
    missing_rows["duplicate_for"] = None
    missing_rows["is_hard"] = False
    # Добавляем в основной DataFrame
    df = pd.concat([df, missing_rows], ignore_index=True)

# Шаг 4: Заполнение новых колонок на основе кластеров
for cluster in clusters.values():
    # Отбираем данные по uuid из кластера
    cluster_data = df[df["uuid"].isin(cluster)].copy()
    # Если есть отсутствующие uuid в df, создадим записи для них
    missing_in_df = set(cluster) - set(cluster_data["uuid"])
    if missing_in_df:
        # Создаем записи для отсутствующих uuid
        missing_rows = pd.DataFrame({"uuid": list(missing_in_df)})
        # Заполняем недостающие колонки
        missing_rows["created"] = pd.NaT  # Нет данных о дате создания
        missing_rows["link"] = None
        missing_rows["is_duplicate"] = False
        missing_rows["duplicate_for"] = None
        missing_rows["is_hard"] = False
        # Добавляем в cluster_data и df
        cluster_data = pd.concat([cluster_data, missing_rows], ignore_index=True)
        df = pd.concat([df, missing_rows], ignore_index=True)
    # Сортируем cluster_data по 'created', NaT будут в конце
    cluster_data = cluster_data.sort_values(by="created")
    original_uuid = cluster_data.iloc[0]["uuid"]  # Самое раннее видео считаем оригиналом
    # Если кластер больше двух элементов, ставим метку is_hard_tmk для всех и не проставляем дубликаты
    if len(cluster) > 2:
        df.loc[df["uuid"].isin(cluster), "is_hard_tmk"] = True
    else:
        # Для кластера из 2 видео проставляем метки дубликатов
        for duplicate_uuid in cluster_data["uuid"].tolist()[1:]:
            df.loc[df["uuid"] == duplicate_uuid, "is_duplicate_tmk"] = True
            df.loc[df["uuid"] == duplicate_uuid, "duplicate_for_tmk"] = original_uuid

# Шаг 5: Проверка корректности заполнения поля duplicate_for_tmk
# Убедимся, что все дубликаты имеют корректного оригинала и что для оригиналов поле duplicate_for_tmk не заполнено
for cluster in clusters.values():
    cluster_data = df[df["uuid"].isin(cluster)].copy()
    # Обрабатываем отсутствующие uuid
    missing_in_df = set(cluster) - set(cluster_data["uuid"])
    if missing_in_df:
        # Создаем записи для отсутствующих uuid
        missing_rows = pd.DataFrame({"uuid": list(missing_in_df)})
        # Заполняем недостающие колонки
        missing_rows["created"] = pd.NaT
        missing_rows["link"] = None
        missing_rows["is_duplicate"] = False
        missing_rows["duplicate_for"] = None
        missing_rows["is_hard"] = False
        # Добавляем в cluster_data
        cluster_data = pd.concat([cluster_data, missing_rows], ignore_index=True)
    # Сортируем cluster_data по 'created', NaT будут в конце
    cluster_data = cluster_data.sort_values(by="created")
    original_uuid = cluster_data.iloc[0]["uuid"]
    # Убедимся, что поле duplicate_for_tmk для оригинала пустое
    assert pd.isna(
        df.loc[df["uuid"] == original_uuid, "duplicate_for_tmk"]
    ).all(), f"Ошибка: у оригинала {original_uuid} заполнено поле duplicate_for_tmk"
    # Убедимся, что у всех дубликатов есть ссылка на оригинал, если кластер состоит из двух видео
    if len(cluster) == 2:
        for duplicate_uuid in cluster_data["uuid"].tolist()[1:]:
            duplicate_for = df.loc[df["uuid"] == duplicate_uuid, "duplicate_for_tmk"].values[0]
            assert (
                duplicate_for == original_uuid
            ), f"Ошибка: у дубликата {duplicate_uuid} неверный оригинал {duplicate_for}, должно быть {original_uuid}"

print("Проверка заполнения полей завершена успешно.")

# Шаг 6: Корректное сравнение колонок, включая NaN и None
df["correct"] = (df["is_duplicate"] == df["is_duplicate_tmk"]) & (
    df["duplicate_for"].fillna("NaN") == df["duplicate_for_tmk"].fillna("NaN")
)

# Шаг 7: Вычисление F1-метрики для поля `duplicate_for` и `duplicate_for_tmk`
# Приводим None и NaN к одному виду (например, NaN)
df["duplicate_for"] = df["duplicate_for"].fillna(np.nan)
df["duplicate_for_tmk"] = df["duplicate_for_tmk"].fillna(np.nan)

# Создаем бинарные метки: 1 - duplicate, 0 - not duplicate
true_labels = df["duplicate_for"].notna().astype(int)
pred_labels = df["duplicate_for_tmk"].notna().astype(int)

# Вычисляем F1-метрику
f1_duplicate_for = f1_score(true_labels, pred_labels, zero_division=0)

# Печать F1 Score для duplicate_for
print(f"F1 score для duplicate_for: {f1_duplicate_for}")

# Шаг 8: Поиск расхождений
discrepancies = df[
    (df["is_duplicate"] != df["is_duplicate_tmk"]) | (df["duplicate_for"].fillna("NaN") != df["duplicate_for_tmk"].fillna("NaN"))
]

# Сохранение расхождений для анализа
discrepancies.to_csv("discrepancies_for_analysis.csv", index=False)

# Сохранение DataFrame с новыми колонками
df.to_csv("updated_train_with_tmk_duplicates.csv", index=False)

### ищем видосы которые дубли на не нашлись TMK, прогоняем их вручную для теста.

In [ ]:
# Шаг 9: Проверка похожести эмбеддингов с помощью tmk-compare-two-tmks
def compare_two_tmks(tmk1_path, tmk2_path):
    """Функция для сравнения двух .tmk файлов с помощью команды tmk-compare-two-tmks.
    Возвращает True, если эмбеддинги похожи, False если нет, и None в случае ошибки.
    """
    command = f"/jupyter/ThreatExchange/tmk/cpp/tmk-compare-two-tmks {tmk1_path} {tmk2_path}"
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True, timeout=60, check=False)
        output = result.stdout.lower()
        if "similar" in output:
            return True
        elif "not similar" in output:
            return False
        else:
            return None  # Не удалось определить
    except subprocess.TimeoutExpired:
        print(f"Команда для сравнения {tmk1_path} и {tmk2_path} превысила время выполнения.")
        return None
    except Exception as e:
        print(f"Ошибка при сравнении {tmk1_path} и {tmk2_path}: {e}")
        return None


# Читаем расхождения из сохранённого файла
discrepancies = pd.read_csv("discrepancies_for_analysis.csv")

# Фильтруем только те строки, где is_duplicate=True и is_duplicate_tmk=False
to_compare = discrepancies[(discrepancies["is_duplicate"] == True) & (discrepancies["is_duplicate_tmk"] == False)].copy()

# Построим маппинг uuid -> путь к .tmk файлу
# Предполагаем, что все .tmk файлы находятся в /jupyter/dup/emb/ и его поддиректориях
tmk_files = glob.glob("/jupyter/emb/*.tmk", recursive=True)
uuid_to_path = {}
for file in tmk_files:
    basename = os.path.basename(file)
    uuid = os.path.splitext(basename)[0]
    uuid_to_path[uuid] = file

# Добавляем столбцы с путями к .tmk файлам
to_compare["tmk1_path"] = to_compare["uuid"].map(uuid_to_path)
to_compare["tmk2_path"] = to_compare["duplicate_for"].map(uuid_to_path)

# Проверяем, что пути найдены
missing_tmks = to_compare[to_compare["tmk1_path"].isna() | to_compare["tmk2_path"].isna()]
if not missing_tmks.empty:
    print("Не найдены пути к некоторым .tmk файлам:")
    print(missing_tmks[["uuid", "duplicate_for"]])
    # Удаляем строки с отсутствующими файлами из дальнейшей обработки
    to_compare = to_compare.dropna(subset=["tmk1_path", "tmk2_path"])


# Функция для сравнения и получения результата
def compare_and_record(row):
    tmk1 = row["tmk1_path"]
    tmk2 = row["tmk2_path"]
    similarity = compare_two_tmks(tmk1, tmk2)
    return similarity


# Применяем функцию ко всем строкам и сохраняем результат
to_compare["tmk_similarity"] = to_compare.apply(compare_and_record, axis=1)

# Сохраняем результаты сравнения
to_compare.to_csv("tmk_similarity_results.csv", index=False)

print("Проверка похожести эмбеддингов завершена. Результаты сохранены в 'tmk_similarity_results.csv'.")

In [ ]:
to_compare

### команды 
/jupyter/ThreatExchange/tmk/cpp/tmk-two-level-score-parallel --c1 0.99 --c2 0.0 *.tmk | sort -n

find . -name '*.tmk' | /jupyter/ThreatExchange/tmk/cpp/tmk-clusterize-parallel -s --min 2 -i

/jupyter/ThreatExchange/tmk/cpp/tmk-compare-two-tmks misc-shelf-sd.tmk misc-shelf-sd.tmk

### отображение 2 видосов

In [ ]:
import asyncio
import os

import ipywidgets as widgets
import nest_asyncio
import pandas as pd
from IPython.display import HTML

# Активация nest-asyncio для работы с асинхронными вызовами в Jupyter
nest_asyncio.apply()

# Загрузка DataFrame 'to_compare'
to_compare = pd.read_csv("tmk_similarity_results.csv")

# Инициализация текущего индекса
current_index = 0

# Создание основного контейнера для кнопок и видео
output = widgets.Output()


# Асинхронная функция для отображения пар видео и кнопок
async def display_video_pair(idx):
    with output:
        output.clear_output(wait=True)  # Очищаем вывод для обновления

        if idx < 0 or idx >= len(to_compare):
            display(HTML("<p>Индекс вне диапазона.</p>"))
            return

        row = to_compare.iloc[idx]
        uuid = row["uuid"]
        duplicate_for = row["duplicate_for"]

        video1_path = f"train_dataset/{uuid}.mp4"
        video2_path = f"train_dataset/{duplicate_for}.mp4"

        # Проверка существования видеофайлов
        video1_exists = os.path.exists(video1_path)
        video2_exists = os.path.exists(video2_path)

        if not video1_exists or not video2_exists:
            missing = []
            if not video1_exists:
                missing.append(f"Оригинал: {uuid}.mp4")
            if not video2_exists:
                missing.append(f"Дубликат: {duplicate_for}.mp4")
            missing_str = ", ".join(missing)
            display(HTML(f"<p style='color:red;'>Видео не найдены: {missing_str}</p>"))
            return

        # Получение информации о дубликате
        is_duplicate = row.get("is_duplicate", "N/A")
        is_duplicate_tmk = row.get("is_duplicate_tmk", "N/A")
        tmk_similarity = row.get("tmk_similarity", "N/A")

        # HTML-код для отображения видео бок о бок с дополнительной информацией
        html_code = f"""
        <div style="display: flex; justify-content: space-around; align-items: center;">
            <div style="text-align: center;">
                <video width="400" controls>
                    <source src="{video1_path}" type="video/mp4">
                    Ваш браузер не поддерживает тег video.
                </video>
                <p><strong>UUID:</strong> {uuid}</p>
                <p><strong>is_duplicate:</strong> {is_duplicate}</p>
                <p><strong>is_duplicate_tmk:</strong> {is_duplicate_tmk}</p>
            </div>
            <div style="text-align: center;">
                <video width="400" controls>
                    <source src="{video2_path}" type="video/mp4">
                    Ваш браузер не поддерживает тег video.
                </video>
                <p><strong>duplicate_for:</strong> {duplicate_for}</p>
                <p><strong>tmk_similarity:</strong> {tmk_similarity}</p>
            </div>
        </div>
        <div style="text-align: center; margin-top: 20px;">
            <p>Пара: {idx + 1} из {len(to_compare)}</p>
        </div>
        """
        display(HTML(html_code))

    # Создание кнопок
    previous_button = widgets.Button(description="⟵ Предыдущий", button_style="info")
    next_button = widgets.Button(description="Следующий ⟶", button_style="info")

    # Асинхронные обработчики нажатий кнопок
    async def on_previous_clicked(b):
        global current_index
        if current_index > 0:
            current_index -= 1
            await display_video_pair(current_index)

    async def on_next_clicked(b):
        global current_index
        if current_index < len(to_compare) - 1:
            current_index += 1
            await display_video_pair(current_index)

    # Привязка функций к кнопкам через асинхронную обработку
    previous_button.on_click(lambda b: asyncio.ensure_future(on_previous_clicked(b)))
    next_button.on_click(lambda b: asyncio.ensure_future(on_next_clicked(b)))

    # Создание контейнера для кнопок
    button_box = widgets.HBox([previous_button, next_button])

    # Отображение кнопок и видео
    display(button_box)


# Отображение контейнера для кнопок и видео
display(output)

# Отображение первой пары видео
await display_video_pair(current_index)

In [ ]:
import os

import ipywidgets as widgets
import nest_asyncio
import pandas as pd
from IPython.display import display

# Активация nest-asyncio для работы с асинхронными вызовами в Jupyter
nest_asyncio.apply()

# Загрузка DataFrame 'to_compare'
to_compare = pd.read_csv("tmk_similarity_results.csv")

# Инициализация текущего индекса
current_index = 0

# Основной контейнер для отображения видео
video_output = widgets.Output()

# Создание кнопок
previous_button = widgets.Button(description="⟵ Предыдущий", button_style="info")
next_button = widgets.Button(description="Следующий ⟶", button_style="info")


# Функция для отображения видео и кнопок в HTML
def generate_html_video_pair(idx):
    # Получаем данные для текущей пары видео
    row = to_compare.iloc[idx]
    uuid = row["uuid"]
    duplicate_for = row["duplicate_for"]

    video1_path = f"train_dataset/{uuid}.mp4"
    video2_path = f"train_dataset/{duplicate_for}.mp4"

    # Проверка существования файлов
    video1_exists = os.path.exists(video1_path)
    video2_exists = os.path.exists(video2_path)

    if not video1_exists or not video2_exists:
        missing = []
        if not video1_exists:
            missing.append(f"Оригинал: {uuid}.mp4")
        if not video2_exists:
            missing.append(f"Дубликат: {duplicate_for}.mp4")
        missing_str = ", ".join(missing)
        return f"<p style='color:red;'>Видео не найдены: {missing_str}</p>"

    # Получаем дополнительные данные
    is_duplicate = row.get("is_duplicate", "N/A")
    is_duplicate_tmk = row.get("is_duplicate_tmk", "N/A")
    tmk_similarity = row.get("tmk_similarity", "N/A")

    # HTML для видео
    html_code = f"""
    <div style="text-align: center;">
        <div style="display: flex; justify-content: space-around; align-items: center;">
            <div>
                <video width="400" controls>
                    <source src="{video1_path}" type="video/mp4">
                    Ваш браузер не поддерживает тег video.
                </video>
                <p><strong>UUID:</strong> {uuid}</p>
                <p><strong>is_duplicate:</strong> {is_duplicate}</p>
                <p><strong>is_duplicate_tmk:</strong> {is_duplicate_tmk}</p>
            </div>
            <div>
                <video width="400" controls>
                    <source src="{video2_path}" type="video/mp4">
                    Ваш браузер не поддерживает тег video.
                </video>
                <p><strong>duplicate_for:</strong> {duplicate_for}</p>
                <p><strong>tmk_similarity:</strong> {tmk_similarity}</p>
            </div>
        </div>
        <p>Пара: {idx + 1} из {len(to_compare)}</p>
    </div>
    """
    return html_code


# Функция для обновления содержимого видео
def update_video_output(idx):
    with video_output:
        video_output.clear_output(wait=True)  # Очищаем вывод
        html_code = generate_html_video_pair(idx)  # Генерируем новый HTML
        display(HTML(html_code))  # Выводим новый HTML-код


# Функции для обработки переходов
def on_previous_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
        update_video_output(current_index)


def on_next_clicked(b):
    global current_index
    if current_index < len(to_compare) - 1:
        current_index += 1
        update_video_output(current_index)


# Привязка обработчиков к кнопкам
previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

# Инициализация интерфейса кнопок
button_box = widgets.HBox([previous_button, next_button])

# Отображение кнопок и видео
display(button_box)  # Отображаем кнопки
display(video_output)  # Отображаем контейнер для видео

# Инициализация отображения первой пары видео
update_video_output(current_index)